In [1]:
import sqlite3 as sq
import pandas as pd

Создание соединения с базой данных

In [2]:
with sq.connect('data/checking-logs.sqlite') as conn:
    conn.cursor()

Получение схемы таблицы test

In [3]:
pd.io.sql.read_sql("PRAGMA table_info(test)", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


Получение первых 10 строк таблицы test

In [4]:
pd.io.sql.read_sql("SELECT * FROM test LIMIT 10", conn)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


Найди среди всех пользователей минимальное значение дельты (периода времени от даты первого коммита
пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.

In [5]:
df_min = pd.io.sql.read_sql('''SELECT t.uid, CAST(MIN(julianday(t.first_commit_ts) -
                            julianday(dl.deadlines, 'unixepoch'))*24 as INTEGER) as 'MIN(diff)'
                            FROM test as t
                            LEFT JOIN deadlines as dl
                            ON t.labname = dl.labs
                            WHERE t.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")''',
                            conn)
df_min.to_sql('df_min', conn, if_exists='replace')
df_min

,uid,MIN(diff)
0,user_30,-202


Найди среди всех пользователей максимальное значение дельты (периода времени от даты первого коммита
пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.

In [6]:
df_max = pd.io.sql.read_sql('''SELECT t.uid, CAST(MAX(julianday(t.first_commit_ts) -
                            julianday(dl.deadlines, 'unixepoch'))*24 as INTEGER) as 'MAX(diff)'
                            FROM test as t
                            LEFT JOIN deadlines as dl
                            ON t.labname = dl.labs
                            WHERE t.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")''',
                            conn)
df_max.to_sql('df_max', conn, if_exists='replace')
df_max

,uid,MAX(diff)
0,user_25,-2


Найди среди всех пользователей среднее значение дельты (периода времени от даты первого коммита
пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.

In [7]:
df_avg = pd.io.sql.read_sql('''SELECT AVG(CAST((julianday(t.first_commit_ts) - 
                            julianday(dl.deadlines, 'unixepoch'))*24 as INTEGER)) as 'AVG(diff)'
                            FROM test as t
                            LEFT JOIN deadlines as dl
                            ON t.labname = dl.labs
                            WHERE t.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")''',
                            conn)
df_avg.to_sql('df_avg', conn, if_exists='replace')
df_avg

,AVG(diff)
0,-89.125


Рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.

In [8]:
# 1 В первом подзапросе (AS t) происходит вычисление средней разницы во времени между first_commit_ts из таблицы test 
#   и deadlines из таблицы deadlines. Для этого используется функция JulianDay, которая преобразует дату и время в 
#   числовое значение Julian Day, а затем вычисляется разница в часах (переводится в целое число с помощью функции CAST).

# 2 Второй подзапрос (AS p) вычисляет количество просмотров страниц для каждого пользователя из таблицы pageviews.

# 3 Оба подзапроса группируются по идентификатору пользователя (uid).

# 4 Затем основной запрос объединяет результаты обоих подзапросов по идентификатору пользователя с использованием оператора ON.

In [9]:
views_diff = pd.io.sql.read_sql('''
    SELECT DISTINCT t.uid, t.diff_avg, p.pageviews
    FROM (
        SELECT uid, AVG(CAST((JulianDay(test.first_commit_ts) - JulianDay(dl.deadlines, 'unixepoch')) * 24 AS INTEGER)) AS diff_avg
        FROM test
        LEFT JOIN deadlines AS dl ON test.labname = dl.labs
        WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
        GROUP BY uid
    ) AS t
    LEFT JOIN (
        SELECT uid, COUNT(datetime) AS pageviews
        FROM pageviews
        GROUP BY uid
    ) AS p
    ON t.uid = p.uid
''', conn)

views_diff.to_sql('views_diff', conn, if_exists='replace')
views_diff

,uid,diff_avg,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


Используй метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [10]:
views_diff[['diff_avg', 'pageviews']].corr()

,diff_avg,pageviews
diff_avg,1.000000,-0.279736
pageviews,-0.279736,1.000000


Закрыть соединение

In [11]:
conn.close()